In [1]:
#Kütüphaneler
#DL KISMI
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
import numpy as np

#SERVER KISMI
import socket
import pickle
import os

Using TensorFlow backend.


In [ ]:
#MODEL TRAİN KISMI
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import numpy as np
VGG16_model = VGG16()

train_path = 'C:/Users/MUSTAFAAKTAS/Desktop/SonDataSet/Train'
valid_path = 'C:/Users/MUSTAFAAKTAS/Desktop/SonDataSet/Validation'

batch_size = 32

train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size = (224,224),
                                                        classes = ['Ananas','Çilek','Elma','Kayısı','Kiraz','Liçi','Muz','Şeftali'], batch_size=batch_size, shuffle=True)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size = (224,224),
                                                        classes = ['Ananas','Çilek','Elma','Kayısı','Kiraz','Liçi','Muz','Şeftali'], batch_size=batch_size, shuffle=True)

model = Sequential()

for layer in VGG16_model.layers[:-2]:
    model.add(layer)
    
for layer in model.layers:
    layer.trainable = False
    
model.add(Dense(128, activation='relu'))
model.add(Dense(activation="softmax", units=8))

model.compile(Adam(lr = .0001),loss='categorical_crossentropy',metrics=['accuracy'])
model.fit_generator(train_batches, steps_per_epoch = len(train_batches),validation_data = valid_batches, 
                    validation_steps = len(valid_batches), epochs = 10, verbose = 1, shuffle=True)


In [2]:
#Bazı sabitler
i = 1 #Dosya adıyla ilgili.

#NOT FOUND UN INDEXI SABIT OLMALI! Son eleman olarak ata!
NOT_FOUND_CASE_VALUE = 99

#TRESHOLD DEGERI
TRESHOLD = 0.8

#BATCH SİZE
BATCH_SIZE = 1

In [ ]:
HOST = '127.0.0.1'  
PORT = 12345       # 1023 ten büyük bir sayı olsun.

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((HOST, PORT))
    s.listen(5)
    i = 1 #Dosya adıyla ilgili.
    print("başladı")
    
    
    while True:
        conn, addr = s.accept()
        print("kabul etti")
        with conn:
            print(' > Yeni bir Baglanti! ', addr)           
            data = (b'')
            while True:  
                yenidata = conn.recv(57) #yeni datayı al
                if yenidata == b'\x00\x01\x02\x03\x04':
                    fileName = 'yeniResim'+str(i)
                    with open("C:/Users/MUSTAFAAKTAS/Desktop/Indirilenler/"+fileName+".jpeg", "wb") as fh:
                        fh.write(((data))) 
                        i = i + 1 #dosya ismini ayarlamak icin
                        print(" >",fileName," kaydedildi.")
                    break            
                data = data + yenidata #yeni gelen data yı öncekini sonuna ekliyoruz.

            ##########ŞİMDİ DL KISMI##########
            try:
                test_path = 'C:/Users/MUSTAFAAKTAS/Desktop/Indirilenler'
                test_batches = ImageDataGenerator(preprocessing_function = preprocess_input).flow_from_directory(test_path, 
                                    target_size = (224,224), classes = ['Test'], batch_size = BATCH_SIZE, shuffle = False)

                test_labels = {0: 'Pineapple', 1: 'Strawberry', 2: 'Apple', 3: 'Apricot',
                              4: 'Cherry', 5: 'Lychee', 6:'Banana', 7:'Peach'}

                preds = model.predict_generator(test_batches, steps=len(test_batches))
                #preds = preds.tolist()
                print(" > Tahminler: ",preds)

                mask = []
                for i in range(len(preds[0])):
                    mask.append(True) if preds[0][i] > TRESHOLD else mask.append(False)
                    
            except FileNotFoundError:
                print("UYARI: test_path bulunamadı!")
            
            ##########BURADA DA CLASS INI CLIENT A GONDERIYORUZ.##########
            if(os.path.exists("C:/Users/MUSTAFAAKTAS/Desktop/Indirilenler/"+fileName+".jpeg")):
                if any(mask):
                    maxDeger = np.argmax(preds[0])
                    cevap = str(test_labels[maxDeger])
                    conn.send(bytes(cevap.encode('utf-8')))
                    conn.send(bytes("-".encode('utf-8'))) #En son da durmasi icin - yolluyoruz            
                else:
                    conn.sendall(('NOT FOUND!').encode('utf-8'))
                print(" > Resmin class'i Client'a gonderildi ..")

                os.remove("C:/Users/MUSTAFAAKTAS/Desktop/Indirilenler/Test/"+fileName+".jpg") #Tahmin edilen resmi sil.